## Find paper informaion from MAG

In [1]:
import requests
import json
import time
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle


> The api function takes in the paper name once it has been lowercase, remove special characters, and only 1 space in between the words. This is the same as the 'cleantitle' coulum in the dataframe incase you need to play around with the regex more  
The attributes in the parameters returns the author normalized name, ID, paper citation count, and paper estimated citation count.  
You can play around with the attributes you want retunred by looking at the documentation here: https://docs.microsoft.com/en-us/academic-services/project-academic-knowledge/reference-paper-entity-attributes  

> ### Replace the subscription key with your own in the headers



In [79]:
#import df_all to test
df_all=pd.read_csv('master_data.csv')

In [80]:
df_all.head()

,index,forum,title,authors,keywords,dec,dec_txt,dec_confidence,dec_title,bld_ratings,...,pub_counts,citation_counts,est_citation_counts,num_authors,affiliations,pub_counts_yr_adj,citation_counts_yr_adj,est_citation_counts_yr_adj,dec_sen_score,review_sen_score
0,0,HJWHIKqgl,Generative Models and Model Criticism via Opti...,"['Dougal J. Sutherland', 'Hsiao-Yu Tung', 'Hei...",['unsupervised learning'],Accept (Poster),This paper presents two ways that MMDs can be ...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,"[32, 5, 29, 2, 100, 123, 129]","[84, 5, 103, 2, 1116, 685, 2235]","[495, 158, 340, 76, 1035, 9940, 1687]",7.0,"['gmail.com', 'cs.cmu.edu', 'gmail.com', 'gmai...","[24, 5, 22, 2, 63, 120, 109]","[58, 5, 62, 2, 333, 446, 1379]","[387, 158, 555, 76, 942, 12832, 4186]",0.123275,0.161820
1,1,SyK00v5xx,A Simple but Tough-to-Beat Baseline for Senten...,"['Sanjeev Arora', 'Yingyu Liang', 'Tengyu Ma']","['natural language processing', 'unsupervised ...",Accept (Poster),A new method for sentence embedding that is si...,NaN,ICLR committee final decision,"['7: Good paper, accept', '7: Good paper, acce...",...,"[83, 87, 90]","[1097, 515, 346]","[2117, 2287, 3092]",3.0,"['cs.princeton.edu', 'cs.princeton.edu', 'cs.p...","[54, 56, 54]","[128, 158, 98]","[3882, 1848, 2663]",0.529200,0.127357
2,2,Hkg4TI9xl,A Baseline for Detecting Misclassified and Out...,"['Dan Hendrycks', 'Kevin Gimpel']",['computer vision'],Accept (Poster),The paper presents an approach that uses the s...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,"[32, 110]","[68, 954]","[994, 3354]",2.0,"['ttic.edu', 'ttic.edu']","[10, 65]","[10, 147]","[555, 3075]",0.000000,0.173518
3,3,Hk1iOLcle,MS MARCO: A Human-Generated MAchine Reading CO...,"['Tri Nguyen', 'Mir Rosenberg', 'Xia Song', 'J...",[],Reject,Though the dataset is likely to have a large i...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,"[17, 5, 75, 216, 267, 11, 6, 1, 45, 18, 2, 13,...","[0, 0, 0, 2792, 4277, 11, 8, 28292, 56584, 44,...","[395, 717, 2479, 16701, 12333, 1762, 405, 3340...",15.0,"['microsoft.com', 'microsoft.com', 'microsoft....","[12, 1, 50, 224, 157, 15, 6, 1, 29, 8, 2, 3, 1...","[22, 1, 128, 2054, 913, 35, 8, 1, 63, 8, 2, 3,...","[364, 322, 1799, 36634, 11432, 644, 407, 322, ...",0.100625,0.185682
4,4,H178hw9ex,Dynamic Steerable Frame Networks,"['Jörn-Henrik Jacobsen', 'Bert De Brabandere',...","['computer vision', 'deep learning']",Reject,This paper studies how to incorporate local in...,NaN,ICLR committee final decision,"['5: Marginally below acceptance threshold', '...",...,"[23, 22, 81]","[61, 36, 72]","[286, 761, 7895]",3.0,"['uva.nl', 'esat.kuleuven.be', 'uva.nl']","[4, 10, 59]","[4, 12, 227]","[175, 626, 7272]",0.069817,0.149944


In [82]:
df_all.columns

Index(['index', 'forum', 'title', 'authors', 'keywords', 'dec', 'dec_txt',
       'dec_confidence', 'dec_title', 'bld_ratings', 'bld_text', 'bld_title',
       'year', 'clean_title', 'author_ids', 'pub_counts', 'citation_counts',
       'est_citation_counts', 'num_authors', 'affiliations',
       'pub_counts_yr_adj', 'citation_counts_yr_adj',
       'est_citation_counts_yr_adj', 'dec_sen_score', 'review_sen_score'],
      dtype='object')

In [81]:
#query each paper on microsoft academic to get author ids
#code to obtain the citation count for each paper under the author 
def obtain_paper(name):
        normalized = name
        url = 'https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate'
        params = {
#             'expr': "Composite(Ti=='"+normalized+"')",
            'expr': "Ti='"+normalized+"'",
            'count': 1,
            'attributes': 'AA.AuN,AA.AuId,CC,ECC,FamId,Id,VFN,Pt,Y,PB,AA.AfN,AA.S'
        }
        headers = {'Ocp-Apim-Subscription-Key': 'a3c6bb2ad6a7483eaf3f132e084b0cc9'}
        r = requests.get(url, params=params, headers=headers)
#         print(r.url)
        result = r.json()
        return result

In [88]:
df_all.shape

(5033, 25)

In [89]:
#filtering based on rows without title form openreview
df_all = df_all[df_all['clean_title'].notna()]

In [90]:
df_all.shape

(5032, 25)

In [91]:
#lowercase, replace special characters, only 1 space between words
papers=df_all['clean_title'].tolist()

In [92]:
#obtain query for all papers
paper_query=[]

for paper in papers:
    paper_query.append(obtain_paper(paper))

In [96]:
len(paper_query)

5032

In [111]:
#parse through paper results, creating column for mag_id, author affiliation, and mag paper source
def paper_parse(query):
    
    if len(query['entities'])==0:
        mag_id= 'nan',
        vfn='nan',
        affs=['']
    else:
        #id number from mag
        mag_id=query['entities'][0]['Id']
        #joural or confrence where the paper was found
        if 'VFN' in query['entities'][0]:
            vfn=query['entities'][0]['VFN']
        else:
            vfn='nan'
        #affiliations for the authors
        affs=[]
        for auth in query['entities'][0]['AA']:
            if 'AfN' in auth:
                affs.append(auth['AfN'])
            else:
                affs.append('nan')
       
    
    return mag_id, vfn, affs

In [112]:
paper_parse(paper_query[4])

(2753454496,
 'nan',
 ['university of amsterdam',
  'katholieke universiteit leuven',
  'university of amsterdam'])

In [115]:
mag_id_list=[]
vfn_list=[]
affs_list=[]

for query in paper_query:
    mag_id,vfn,affs=paper_parse(query)
    #add to master list
    mag_id_list.append(mag_id)
    vfn_list.append(vfn)
    affs_list.append(affs)

In [122]:
#add to main dataframe
df_all['mag_id']=mag_id_list
df_all['mag_venue']=vfn_list
df_all['mag_auth_affiliations']=affs_list

In [123]:
df_all.head()

,index,forum,title,authors,keywords,dec,dec_txt,dec_confidence,dec_title,bld_ratings,...,num_authors,affiliations,pub_counts_yr_adj,citation_counts_yr_adj,est_citation_counts_yr_adj,dec_sen_score,review_sen_score,mag_id,mag_venue,mag_auth_affiliations
0,0,HJWHIKqgl,Generative Models and Model Criticism via Opti...,"['Dougal J. Sutherland', 'Hsiao-Yu Tung', 'Hei...",['unsupervised learning'],Accept (Poster),This paper presents two ways that MMDs can be ...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,7.0,"['gmail.com', 'cs.cmu.edu', 'gmail.com', 'gmai...","[24, 5, 22, 2, 63, 120, 109]","[58, 5, 62, 2, 333, 446, 1379]","[387, 158, 555, 76, 942, 12832, 4186]",0.123275,0.161820,2950863313,arXiv: Machine Learning,"[carnegie mellon university, carnegie mellon u..."
1,1,SyK00v5xx,A Simple but Tough-to-Beat Baseline for Senten...,"['Sanjeev Arora', 'Yingyu Liang', 'Tengyu Ma']","['natural language processing', 'unsupervised ...",Accept (Poster),A new method for sentence embedding that is si...,NaN,ICLR committee final decision,"['7: Good paper, accept', '7: Good paper, acce...",...,3.0,"['cs.princeton.edu', 'cs.princeton.edu', 'cs.p...","[54, 56, 54]","[128, 158, 98]","[3882, 1848, 2663]",0.529200,0.127357,2752172973,International Conference on Learning Represent...,"[princeton university, princeton university, p..."
2,2,Hkg4TI9xl,A Baseline for Detecting Misclassified and Out...,"['Dan Hendrycks', 'Kevin Gimpel']",['computer vision'],Accept (Poster),The paper presents an approach that uses the s...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,2.0,"['ttic.edu', 'ttic.edu']","[10, 65]","[10, 147]","[555, 3075]",0.000000,0.173518,2952053192,arXiv: Neural and Evolutionary Computing,"[toyota technological institute at chicago, to..."
3,3,Hk1iOLcle,MS MARCO: A Human-Generated MAchine Reading CO...,"['Tri Nguyen', 'Mir Rosenberg', 'Xia Song', 'J...",[],Reject,Though the dataset is likely to have a large i...,NaN,ICLR committee final decision,"['6: Marginally above acceptance threshold', '...",...,15.0,"['microsoft.com', 'microsoft.com', 'microsoft....","[12, 1, 50, 224, 157, 15, 6, 1, 29, 8, 2, 3, 1...","[22, 1, 128, 2054, 913, 35, 8, 1, 63, 8, 2, 3,...","[364, 322, 1799, 36634, 11432, 644, 407, 322, ...",0.100625,0.185682,2951534261,arXiv: Computation and Language,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,4,H178hw9ex,Dynamic Steerable Frame Networks,"['Jörn-Henrik Jacobsen', 'Bert De Brabandere',...","['computer vision', 'deep learning']",Reject,This paper studies how to incorporate local in...,NaN,ICLR committee final decision,"['5: Marginally below acceptance threshold', '...",...,3.0,"['uva.nl', 'esat.kuleuven.be', 'uva.nl']","[4, 10, 59]","[4, 12, 227]","[175, 626, 7272]",0.069817,0.149944,2753454496,nan,"[university of amsterdam, katholieke universit..."


In [131]:
df_all['mag_auth_affiliations'].explode().str.lower().value_counts(9)

nan                                                   0.455361
google                                                0.076451
university of california berkeley                     0.021039
carnegie mellon university                            0.019265
massachusetts institute of technology                 0.017338
                                                        ...   
medical university of graz                            0.000051
wesleyan university                                   0.000051
indian institute of technology delhi                  0.000051
beijing university of posts and telecommunications    0.000051
indian institute of technology kharagpur              0.000051
Name: mag_auth_affiliations, Length: 591, dtype: float64

In [128]:
#save updated dataframe to csv
df_all.to_csv('master_data.csv')
